In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix
from pyspark.ml.feature import Bucketizer

df_yield = spark.read.format("jdbc") \
                     .option("driver","com.mysql.cj.jdbc.Driver") \
                     .option("url","jdbc:mysql://116.204.183.76:13306/yield") \
                     .option("user","cube_user") \
                     .option("password","cu6euser") \
                     .option("dbtable","yield_data") \
                     .load()

In [18]:
df_yield.count()

28242

In [19]:
df_yield.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37
5,Albania,Wheat,1990,30197,1485.0,121.0,16.37
6,Albania,Maize,1991,29068,1485.0,121.0,15.36
7,Albania,Potatoes,1991,77818,1485.0,121.0,15.36
8,Albania,"Rice, paddy",1991,28538,1485.0,121.0,15.36
9,Albania,Sorghum,1991,6667,1485.0,121.0,15.36


In [20]:
bucketize = Bucketizer(
    splits=[0, 25000, 50000, float('Inf')],
    inputCol="hg/ha_yield",
    outputCol="yield_name"
)

In [21]:
df_wheat = df_yield[df_yield['item'] == 'Wheat']

In [22]:
df_bin = bucketize.transform(df_wheat)

In [23]:
df_bin.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,yield_name
0,Albania,Wheat,1990,30197,1485.0,121.00,16.37,1.0
1,Albania,Wheat,1991,20698,1485.0,121.00,15.36,0.0
2,Albania,Wheat,1992,24388,1485.0,121.00,16.06,0.0
3,Albania,Wheat,1993,29976,1485.0,121.00,16.05,1.0
4,Albania,Wheat,1994,24745,1485.0,201.00,16.96,0.0
5,Albania,Wheat,1995,28703,1485.0,251.00,15.67,1.0
6,Albania,Wheat,1996,21741,1485.0,313.96,15.64,0.0
7,Albania,Wheat,1997,28516,1485.0,376.93,15.90,1.0
8,Albania,Wheat,1998,28037,1485.0,439.89,16.27,1.0
9,Albania,Wheat,1999,24954,1485.0,502.86,16.57,0.0


In [35]:
df_bin = df_bin.withColumnRenamed('yield_name', 'yield_range')

In [36]:
df_bin.printSchema()

root
 |-- Area: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- hg/ha_yield: integer (nullable = true)
 |-- average_rain_fall_mm_per_year: double (nullable = true)
 |-- pesticides_tonnes: double (nullable = true)
 |-- avg_temp: double (nullable = true)
 |-- yield_range: double (nullable = true)



In [47]:
df_bin.select('yield_range').distinct().toPandas()

,yield_range
0,0.0
1,1.0
2,2.0


In [48]:
df_bin.select('Year').distinct().sort('Year').toPandas()

,Year
0,1990
1,1991
2,1992
3,1993
4,1994
5,1995
6,1996
7,1997
8,1998
9,1999


In [49]:
df_train = df_bin[df_bin.Year <= 2010]
df_test  = df_bin[df_bin.Year > 2010]

In [50]:
df_train.count()

3346

In [52]:
df_test.limit(10).toPandas()

Exception ignored in: <function JavaWrapper.__del__ at 0x7f7c80181090>          
Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f7c80181090>
Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'


,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,yield_range
0,Albania,Wheat,2011,42301,1485.0,1302.63,16.45,1.0
1,Albania,Wheat,2012,41096,1485.0,766.25,16.70,1.0
2,Albania,Wheat,2013,41292,1485.0,982.32,17.41,1.0
3,Algeria,Wheat,2011,17405,89.0,13861.76,17.79,0.0
4,Algeria,Wheat,2012,17639,89.0,17379.76,17.91,0.0
5,Algeria,Wheat,2013,19100,89.0,17278.65,17.65,0.0
6,Angola,Wheat,2011,10959,1010.0,40.00,24.15,0.0
7,Angola,Wheat,2012,11258,1010.0,40.00,24.24,0.0
8,Angola,Wheat,2013,9022,1010.0,40.00,24.55,0.0
9,Argentina,Wheat,2011,35063,591.0,100424.62,17.58,1.0


In [58]:
 assemble = VectorAssembler(
    inputCols=df_bin.columns[4:7],
    outputCol="features"
 )

In [59]:
df_train = assemble.transform(df_train)
df_test  = assemble.transform(df_test)

In [60]:
df_train.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,yield_range,features
0,Albania,Wheat,1990,30197,1485.0,121.00,16.37,1.0,"[1485.0, 121.0, 16.37]"
1,Albania,Wheat,1991,20698,1485.0,121.00,15.36,0.0,"[1485.0, 121.0, 15.36]"
2,Albania,Wheat,1992,24388,1485.0,121.00,16.06,0.0,"[1485.0, 121.0, 16.06]"
3,Albania,Wheat,1993,29976,1485.0,121.00,16.05,1.0,"[1485.0, 121.0, 16.05]"
4,Albania,Wheat,1994,24745,1485.0,201.00,16.96,0.0,"[1485.0, 201.0, 16.96]"
5,Albania,Wheat,1995,28703,1485.0,251.00,15.67,1.0,"[1485.0, 251.0, 15.67]"
6,Albania,Wheat,1996,21741,1485.0,313.96,15.64,0.0,"[1485.0, 313.96, 15.64]"
7,Albania,Wheat,1997,28516,1485.0,376.93,15.90,1.0,"[1485.0, 376.93, 15.9]"
8,Albania,Wheat,1998,28037,1485.0,439.89,16.27,1.0,"[1485.0, 439.89, 16.27]"
9,Albania,Wheat,1999,24954,1485.0,502.86,16.57,0.0,"[1485.0, 502.86, 16.57]"


In [61]:
df_train = df_train.withColumn('yield_range',df_train.yield_range.cast("double"))
df_test  = df_test.withColumn('yield_range',df_test.yield_range.cast("double"))

In [62]:
# Building Model Decission Tree
dt = DecisionTreeClassifier(
    featuresCol = 'features',
    labelCol = 'yield_range'
)

model = dt.fit(df_train)

In [63]:
# Predict
df_pred = model.transform(df_test)

In [64]:
df_pred.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,yield_range,features,rawPrediction,probability,prediction
0,Albania,Wheat,2011,42301,1485.0,1302.63,16.45,1.0,"[1485.0, 1302.63, 16.45]","[90.0, 63.0, 46.0]","[0.45226130653266333, 0.3165829145728643, 0.23...",0.0
1,Albania,Wheat,2012,41096,1485.0,766.25,16.70,1.0,"[1485.0, 766.25, 16.7]","[90.0, 63.0, 46.0]","[0.45226130653266333, 0.3165829145728643, 0.23...",0.0
2,Albania,Wheat,2013,41292,1485.0,982.32,17.41,1.0,"[1485.0, 982.32, 17.41]","[90.0, 63.0, 46.0]","[0.45226130653266333, 0.3165829145728643, 0.23...",0.0
3,Algeria,Wheat,2011,17405,89.0,13861.76,17.79,0.0,"[89.0, 13861.76, 17.79]","[769.0, 188.0, 16.0]","[0.7903391572456321, 0.1932168550873587, 0.016...",0.0
4,Algeria,Wheat,2012,17639,89.0,17379.76,17.91,0.0,"[89.0, 17379.76, 17.91]","[769.0, 188.0, 16.0]","[0.7903391572456321, 0.1932168550873587, 0.016...",0.0
5,Algeria,Wheat,2013,19100,89.0,17278.65,17.65,0.0,"[89.0, 17278.65, 17.65]","[769.0, 188.0, 16.0]","[0.7903391572456321, 0.1932168550873587, 0.016...",0.0
6,Angola,Wheat,2011,10959,1010.0,40.00,24.15,0.0,"[1010.0, 40.0, 24.15]","[173.0, 9.0, 0.0]","[0.9505494505494505, 0.04945054945054945, 0.0]",0.0
7,Angola,Wheat,2012,11258,1010.0,40.00,24.24,0.0,"[1010.0, 40.0, 24.24]","[173.0, 9.0, 0.0]","[0.9505494505494505, 0.04945054945054945, 0.0]",0.0
8,Angola,Wheat,2013,9022,1010.0,40.00,24.55,0.0,"[1010.0, 40.0, 24.55]","[173.0, 9.0, 0.0]","[0.9505494505494505, 0.04945054945054945, 0.0]",0.0
9,Argentina,Wheat,2011,35063,591.0,100424.62,17.58,1.0,"[591.0, 100424.62, 17.58]","[19.0, 14.0, 0.0]","[0.5757575757575758, 0.42424242424242425, 0.0]",0.0


In [66]:
# Check Performance
evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='yield_range'
)

evaluator.evaluate(df_pred)

0.6354948899240829